In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

2023-04-29 11:05:01.545896: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 11:05:01.755598: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-29 11:05:01.762414: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-29 11:05:01.762441: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

# Transformers

## Chargement des Données

In [2]:
# Données d'entrainement
train_data_complete = pd.read_csv("../data/allocine_genres_train.csv", sep=",")
train_data = train_data_complete[["titre", "synopsis", "genre"]]

# Données de test/validation
test_data_complete = pd.read_csv("../data/allocine_genres_test.csv", sep=",")
test_data = test_data_complete[["titre", "synopsis", "genre"]]

In [3]:
# Liste de classes et ajout d'un identifiant numérique pour chaque classe
genre_name = sorted(train_data.genre.unique().flatten())
print("Genres:", genre_name)
print("Nombre d'exemplaires:", len(train_data))

label2id = {genre_name[i]:i for i in range(len(genre_name))}
id2label = {i:genre_name[i] for i in range(len(genre_name))}

Genres: ['biopic', 'comédie', 'documentaire', 'drame', 'historique', 'horreur', 'policier', 'romance', 'science fiction']
Nombre d'exemplaires: 2875


In [4]:
# Proportion des données qui sera utilisée
scale = 0.2

In [5]:
from datasets import Features, Value, ClassLabel, Dataset, DatasetDict

data_df = pd.DataFrame()
# Chaque film est décrit par un titre et un synopsis
split_titre = train_data.titre
data_df["text"] = split_titre + " " + train_data.synopsis
# Chaque filme possède un attribut qui décrit son genre (auquel a été associé un identifiant numérique)
data_df["genre"] = train_data.genre.map(label2id)

# Transformation du DataFrame en objet de type Dataset utilisé par HuggingFace
province_features = Features({"text": Value('string'),
                              "genre": ClassLabel(names=genre_name)})
data = Dataset.from_pandas(data_df, features=province_features)
# Découpage en train et test
data = data.train_test_split(test_size=0.2, shuffle=True, seed=12)

In [6]:
data["train"].features

{'text': Value(dtype='string', id=None),
 'genre': ClassLabel(names=['biopic', 'comédie', 'documentaire', 'drame', 'historique', 'horreur', 'policier', 'romance', 'science fiction'], id=None)}

In [7]:
data["train"][0]

{'text': "True Romance Le jour de son anniversaire , Clarence Worley rencontre la splendide Alabama dans un cinéma miteux . Coup de foudre immédiat . Après une nuit d' amour , Alabama avoue a Clarence qu' elle a été en fait engagée par le patron de Clarence comme cadeau d' anniversaire . De là va commencer une folle aventure .",
 'genre': 7}

## Tokénisation

In [8]:
from transformers import AutoTokenizer

model_ckpt = "baptiste-pasquier/distilcamembert-allocine"

# Chargement du tokéniseur pré-entraîné correspondant au modèle utilisé
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [10]:
# Tokénisation des 2 premières instances
preprocess_function(data["train"][:2])

{'input_ids': [[5, 10205, 35, 6900, 291, 54, 209, 8, 58, 2575, 21, 7, 15195, 2101, 692, 451, 2404, 162, 13, 11072, 114, 10966, 4547, 29, 23, 1545, 4189, 914, 21, 9, 8207, 8, 13268, 8567, 21, 9, 407, 28, 656, 18, 11, 1724, 21, 7, 114, 10966, 4547, 16102, 33, 15195, 2101, 46, 11, 109, 33, 101, 22, 82, 8158, 37, 16, 3656, 8, 15195, 2101, 79, 2275, 18, 11, 2575, 21, 9, 137, 241, 198, 1348, 28, 5907, 3371, 21, 9, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [5, 10094, 4164, 10094, 4164, 21, 7, 338, 134, 21, 7, 27406, 422, 37, 89, 27193, 10, 21, 7, 2656, 15, 58, 2377, 22, 1299, 8, 48, 8840, 8, 58, 17806, 8, 64, 1835, 94, 21, 9, 123, 10672, 108, 16, 12750, 20, 6224, 3446, 21, 7, 109, 82, 16, 10242, 42, 77, 1058, 2940, 8, 1077, 42, 16, 246, 2635, 31, 11156, 13, 438, 8, 13, 1010, 21, 9, 28222, 8, 4532, 21, 7, 44, 2635, 52, 12, 21, 6559, 98, 19216, 21, 7, 16, 40, 6

In [11]:
# Tokenisation de la totalité des données : chaque unité est remplacée par un identifiant numérique
tokenized_data = data.map(preprocess_function, batched=True, batch_size=None)

Map:   0%|          | 0/2300 [00:00<?, ? examples/s]

Map:   0%|          | 0/575 [00:00<?, ? examples/s]

In [12]:
tokenized_data["train"][0]

{'text': "True Romance Le jour de son anniversaire , Clarence Worley rencontre la splendide Alabama dans un cinéma miteux . Coup de foudre immédiat . Après une nuit d' amour , Alabama avoue a Clarence qu' elle a été en fait engagée par le patron de Clarence comme cadeau d' anniversaire . De là va commencer une folle aventure .",
 'genre': 7,
 'input_ids': [5,
  10205,
  35,
  6900,
  291,
  54,
  209,
  8,
  58,
  2575,
  21,
  7,
  15195,
  2101,
  692,
  451,
  2404,
  162,
  13,
  11072,
  114,
  10966,
  4547,
  29,
  23,
  1545,
  4189,
  914,
  21,
  9,
  8207,
  8,
  13268,
  8567,
  21,
  9,
  407,
  28,
  656,
  18,
  11,
  1724,
  21,
  7,
  114,
  10966,
  4547,
  16102,
  33,
  15195,
  2101,
  46,
  11,
  109,
  33,
  101,
  22,
  82,
  8158,
  37,
  16,
  3656,
  8,
  15195,
  2101,
  79,
  2275,
  18,
  11,
  2575,
  21,
  9,
  137,
  241,
  198,
  1348,
  28,
  5907,
  3371,
  21,
  9,
  6,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [13]:
# Affichage des tokens. DistilBERT utilise l'algorithme WordPiece
tokens = tokenizer.convert_ids_to_tokens(tokenized_data["train"][0]['input_ids'])
print(tokenized_data["train"][0]["text"])
print(tokens)

True Romance Le jour de son anniversaire , Clarence Worley rencontre la splendide Alabama dans un cinéma miteux . Coup de foudre immédiat . Après une nuit d' amour , Alabama avoue a Clarence qu' elle a été en fait engagée par le patron de Clarence comme cadeau d' anniversaire . De là va commencer une folle aventure .
['<s>', '▁Tru', 'e', '▁Roman', 'ce', '▁Le', '▁jour', '▁de', '▁son', '▁anniversaire', '▁', ',', '▁Clar', 'ence', '▁W', 'or', 'ley', '▁rencontre', '▁la', '▁splendide', '▁A', 'lab', 'ama', '▁dans', '▁un', '▁cinéma', '▁mit', 'eux', '▁', '.', '▁Coup', '▁de', '▁foudre', '▁immédiat', '▁', '.', '▁Après', '▁une', '▁nuit', '▁d', "'", '▁amour', '▁', ',', '▁A', 'lab', 'ama', '▁avoue', '▁a', '▁Clar', 'ence', '▁qu', "'", '▁elle', '▁a', '▁été', '▁en', '▁fait', '▁engagée', '▁par', '▁le', '▁patron', '▁de', '▁Clar', 'ence', '▁comme', '▁cadeau', '▁d', "'", '▁anniversaire', '▁', '.', '▁De', '▁là', '▁va', '▁commencer', '▁une', '▁folle', '▁aventure', '▁', '.', '</s>', '<pad>', '<pad>', '<pad>',

In [14]:
# Taille du vocabulaire
tokenizer.vocab_size

32005

In [15]:
# Taille de contexte maximum
tokenizer.model_max_length

512

## Préparation de l'Evaluation

In [16]:
import evaluate

accuracy = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    return acc

## Construction et Entraînement du Transformer (PyTorch -> NE FONCTIONNE PAS)

In [18]:
# from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
# import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_ckpt, num_labels=len(genre_name), id2label=id2label, label2id=label2id
# ).to(device)

In [19]:
# training_args = TrainingArguments(
#     output_dir=f"{model_ckpt}-finetuned-genre",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=6,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
# )

In [20]:
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_data["train"],
#     eval_dataset=tokenized_data["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

In [21]:
# trainer.train()

## Construction et Entraînement du Transformer (Tensorflow)

In [22]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [23]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 64
num_epochs = 2
batches_per_epoch = len(tokenized_data["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

2023-04-29 11:05:17.841220: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-29 11:05:17.844504: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-29 11:05:17.844559: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fabien): /proc/driver/nvidia/version does not exist
2023-04-29 11:05:17.852140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=9, id2label=id2label, label2id=label2id
)

ValueError: cannot reshape array of size 1536 into shape (768,9)

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data["train"],
    shuffle=True,
    batch_size=64,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_data["test"],
    shuffle=False,
    batch_size=64,
    collate_fn=data_collator,
)
print(type(tf_validation_set))

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set, label_cols=["genre"])

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[metric_callback])

## Analyse des Résultats

In [ ]:
# Prédictions pour les données de test
preds_output = trainer.predict(tokenized_data['test'])

In [ ]:
preds_output

In [ ]:
preds_output.metrics

In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)
y_valid = tokenized_data['test']['label']
labels = tokenized_data['test'].features['label'].names

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    labels_for_fig = [l[0:4]+'.' for l in labels]
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                                  display_labels=labels_for_fig)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()
    
plot_confusion_matrix(y_preds, y_valid, labels)

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # Fonction qui retourne la perte (entropie croisée) et la classe prédite
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device), 
                             reduction="none")
    return {"loss": loss.cpu().numpy(), 
            "predicted_label": pred_label.cpu().numpy()}

In [ ]:
# Conversion des données au bon format
tokenized_data.set_format("torch", 
                            columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Calcul des valeurs de perte
tokenized_data["test"] = tokenized_data["test"].map(
    forward_pass_with_label, batched=True, batch_size=64)

In [ ]:
# Création d'un DataFrame avec les textes, les pertes les classe (prédites et attendues)

def label_int2str(row):
    return tokenized_data["train"].features["label"].int2str(row)

tokenized_data.set_format("pandas")
cols = ["text", "label", "predicted_label", "loss"]
df_test = tokenized_data["test"][:][cols]
df_test["label"] = df_test["label"].apply(label_int2str)
df_test["predicted_label"] = (df_test["predicted_label"]
                              .apply(label_int2str))

In [ ]:
# Pour éviter l'affichage tronqué des descriptions
pd.set_option('display.max_colwidth', -1)
# Affichage des 10 premières instances triées par perte décroissante
df_test.sort_values("loss", ascending=False).head(10)

In [ ]:
# Affichage des 10 premières instances triées par perte croissante
# Cela permet de voir les instances pour lesquelles les prédictions sont les plus certaines
df_test.sort_values("loss", ascending=True).head(10)